In [24]:
import json

def get_difference(word1, word2, word_list):
    try:
        index1 = word_list.index(word1)
        index2 = word_list.index(word2)
        return index2 - index1
    except ValueError:
        return "One or both words not found in the list"

def update_hierarchy(data, parent_full_name='', root_word ='', parent_id=5):
    if 'horizonal_id' not in data.keys():
        data['horizonal_id'] = parent_id 
    for child in data['children']:
        # Update name
        old_name = child['full_name']
        new_word = ' '.join(word for word in old_name.split() if word not in parent_full_name.split())
        child['name'] = new_word

        # Update horizontal_id
        child['horizonal_id'] = get_difference(root_word, child['name'], old_name.split()) + parent_id

        # Recursive call for child's children
        if 'children' in child:
            update_hierarchy(child, child["full_name"], root_word)

# Load JSON file
with open('hier_input.json', 'r') as f:
    data = json.load(f)

# Update hierarchy
update_hierarchy(data, data['full_name'], data['full_name'])

# Save updated JSON file
with open('output_hier.json', 'w') as f:
    json.dump(data, f, indent=4)

In [33]:
import networkx as nx
import json

def add_edges(graph, data, parent_node=None):
    full_name = data['full_name']
    name = data.get('name', full_name)
    horizontal_id = data.get('horizonal_id', None)

    graph.add_node(name, name=name, full_name=full_name, horizontal_id=horizontal_id)

    if parent_node is not None:
        graph.add_edge(parent_node, name)

    for child in data.get('children', []):
        add_edges(graph, child, parent_node=name)

# Load JSON file
with open('output_hier.json', 'r') as f:
    data = json.load(f)

# Create directed graph
G = nx.DiGraph()

# Add edges to the graph
add_edges(G, data)

# Print nodes with their properties
print("node")
for node in G.nodes(data=True):
    print(node)
print("edge")
for edge in G.edges(data=True):
    print(edge)


node
('world', {'name': 'world', 'full_name': 'world', 'horizontal_id': 5})
('cup', {'name': 'cup', 'full_name': 'world cup', 'horizontal_id': 6})
('first', {'name': 'first', 'full_name': 'first world cup', 'horizontal_id': 4})
('goal', {'name': 'goal', 'full_name': 'first world cup goal', 'horizontal_id': 7})
('own', {'name': 'own', 'full_name': 'first world cup own', 'horizontal_id': 7})
('my', {'name': 'my', 'full_name': 'my first world cup', 'horizontal_id': 3})
edge
('world', 'cup', {})
('cup', 'first', {})
('first', 'goal', {})
('first', 'own', {})
('first', 'my', {})


In [48]:
import pandas as pd
# Convert nodes and edges to dataframes
nodes_df = pd.DataFrame(G.nodes(data=True), columns=['id', 'properties'])
edges_df = pd.DataFrame(G.edges(data=True), columns=['source', 'target', 'properties'])

# Convert dataframes to JSON
nodes_json = nodes_df.to_json(orient='records')
edges_json = edges_df.to_json(orient='records')

print(nodes_json)
print(edges_json)




[{"id":"world","properties":{"name":"world","full_name":"world","horizontal_id":5}},{"id":"cup","properties":{"name":"cup","full_name":"world cup","horizontal_id":6}},{"id":"first","properties":{"name":"first","full_name":"first world cup","horizontal_id":4}},{"id":"goal","properties":{"name":"goal","full_name":"first world cup goal","horizontal_id":7}},{"id":"own","properties":{"name":"own","full_name":"first world cup own","horizontal_id":7}},{"id":"my","properties":{"name":"my","full_name":"my first world cup","horizontal_id":3}}]
[{"source":"world","target":"cup","properties":{}},{"source":"cup","target":"first","properties":{}},{"source":"first","target":"goal","properties":{}},{"source":"first","target":"own","properties":{}},{"source":"first","target":"my","properties":{}}]


In [49]:
import networkx as nx
import numpy as np
import pandas as pd
import pytest
import IPython.display
import pyciemss



def spring_force_graph(
    nodes_json,edges_json):


    schema = pyciemss.visuals.vega.load_schema("word_graph.vg.json")


    schema["data"] = pyciemss.visuals.vega.replace_named_with(
        schema["data"], "node-data", ["values"], nodes_json
    )

    schema["data"] = pyciemss.visuals.vega.replace_named_with(
        schema["data"], "link-data", ["values"], edges_json
    )


    return schema
schema = spring_force_graph(nodes_json,edges_json )
pyciemss.visuals.plots.ipy_display(schema, format="interactive", dpi=72)